# Adaptive and Iterative Mechanism for Differentially Private Synthetic Data

This method is designed for categorical data.

In [ ]:
# stdlib
import warnings
import sys

warnings.filterwarnings("ignore")


# synthcity absolute
from synthcity.plugins import Plugins
from synthcity.utils.datasets.categorical.categorical_adult import CategoricalAdultDataloader
import synthcity.logger as log
log.add(sink=sys.stderr, level="INFO")

eval_plugin = "aim"

### Load dataset

In [ ]:
# synthcity absolute
from synthcity.plugins.core.dataloader import GenericDataLoader

X = CategoricalAdultDataloader().load()
loader = GenericDataLoader(X, target_column="income>50K", sensitive_columns=["sex", "race"])

loader.dataframe()

### Train the generator

In [ ]:
# synthcity absolute
from synthcity.plugins import Plugins

syn_model = Plugins().get(eval_plugin)


In [ ]:

syn_model.fit(loader)

### Generate new samples

In [ ]:
syn_model.generate(count=10).dataframe()

In [ ]:
# third party
import matplotlib.pyplot as plt

syn_model.plot(plt, loader, count=100)

plt.show()

### Benchmarks

In [ ]:
# synthcity absolute
from synthcity.benchmark import Benchmarks

score = Benchmarks.evaluate(
    [
        (eval_plugin, eval_plugin, {"epsilon": 1.0, "delta": 1e-7, "max_model_size": 80, "degree": 2, "num_marginals": None, "max_cells": 1000}),
    ],  # (testname, plugin, plugin_args) The plugin_args are given are simply to illustrate some of the paramters that can be passed to the plugin
    loader,
    repeats=2,
    metrics={
        "detection": ["detection_mlp"],
        "privacy": ["distinct l-diversity", "k-anonymization", "k-map"],
    },
)

In [ ]:
Benchmarks.print(score)